In [1]:
# use svg graphics, display inline
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import glob
import re
import copy
import sys

# basic scientific computing imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from sparklines import sparklines

# chatbot
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain, SQLDatabaseChain
from langchain.utilities import PythonREPL
import openai
from seaborn.utils import load_dataset

from tqdm.notebook import trange, tqdm

# hex colors for plotting
SOFT_PURPLE = '#8565C4'
SOFT_RED = '#C23F38'
SOFT_GREEN = '#56B000'
NEUTRAL_GREY = '#A9A9A9'

# display config
pd.set_option('display.float_format', lambda x: '%.3f' % x)
plt.rcParams['figure.figsize'] = 6, 4
plt.style.use('ggplot')
np.set_printoptions(suppress=True)
np.random.seed(42)

print(sys.version)

3.9.16 | packaged by conda-forge | (main, Feb  1 2023, 21:38:11) 
[Clang 14.0.6 ]


In [ ]:
def print_first_and_best(df):
    
    best_row_number = pd.to_numeric(df.iloc[1:].rating, errors='coerce').idxmax() + 1
    best_row = df.iloc[1:].sort_values('rating', ascending=False).iloc[0]   # can't be the first one
    
    print(f"First Story\n\nPrompt: {df.iloc[0]['prompt']}\n\nStory: {df.iloc[0]['story']}")
    print('\n###########\n')
    print(f"Best ({best_row_number + 1}) Story\n\nPrompt: {best_row['prompt']}\n\nStory: {best_row['story']}")

# Define models, prompts, and pipeline

In [21]:
llm_language = ChatOpenAI(
    model_name='gpt-3.5-turbo', 
    temperature=0.3, 
    max_tokens = 2048
)

prompt_generator_prompt = PromptTemplate(
    input_variables=['task'],
    template='''
Context:
An AI model must be correctly prompted to generate high quality output. 

Your job is to generate a prompt for the following task: {task}

Generated prompt:
'''
)

story_generator_prompt = PromptTemplate(
    input_variables=['prompt'],
    template='{prompt}'
)

rater_prompt = PromptTemplate(
    input_variables=['story', 'optimize_for'],
    template='''
Rate the following story on a scale from 1-100. The rating should reflect the story's {optimize_for}.
After giving the story a numerical rating, give a brief suggestion for what could make it better. The output should be in the form <rating>: <suggestions>

Story:
{story}

Rating and suggestions:
'''
)

new_prompt_from_feedback_prompt = PromptTemplate(
    input_variables=['task', 'prompt', 'rating', 'suggestions', 'optimize_for'],
    template='''
Context:
An AI model must be correctly prompted to generate high quality output. 

Your job is to generate a prompt for the following task: {task}

Below is a prompt that generated story with a score of {rating} in terms of {optimize_for}. We want to make it a 100:
{prompt}

Here is a suggestion to make it better:
{suggestions}

Incorporate this feedback into a new prompt to encourage the AI to write better story. Be sure to keep your feedback concise, really focus on 2-3 key suggestions. If the last story was rated 100, make it a little bit better.

New prompt:
'''
)

In [48]:
def generate_stories(initial_input, optimize_for, llm, n_iterations=5):

    data = []
    prompt = initial_input
    
    story_chain = LLMChain(llm=llm, prompt=story_generator_prompt)
    judge_chain = LLMChain(llm=llm, prompt=rater_prompt)
    new_prompt_chain = LLMChain(llm=llm, prompt=new_prompt_from_feedback_prompt)
    
    for i in trange(n_iterations):   # trange just adds a little progress bar
        story = story_chain.run(prompt)

        rating = judge_chain.run({
            'story': story, 
            'optimize_for': optimize_for
        })

        # extract the rating and suggestions
        # might not need to parse the string, just feed the response to the rewriter
        numerical_rating, suggestions = rating.split(': ', 1)

        # log the results
        data.append({
            'task': initial_input, 
            'prompt': prompt,
            'story': story,
            'rating': numerical_rating, 
            'suggestions': suggestions,
        })

        prompt = new_prompt_chain.run({
            'task': initial_input, 
            'prompt': prompt, 
            'rating': numerical_rating, 
            'suggestions': suggestions,
            'optimize_for': optimize_for
        })

    return pd.DataFrame(data)

# Scary Story

In [31]:
initial_input = 'Write a scary story. Be sure to specify that the story should be no longer than 10 sentences'
optimize_for = 'scariness, surprise, literary quality'

df_scary_stories = generate_stories(
    initial_input=initial_input,
    optimize_for=optimize_for,
    llm=llm_language,
    n_iterations=10
)

print_first_and_best(df_scary_stories)

  0%|          | 0/10 [00:00<?, ?it/s]

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')).


First Story

Prompt: Write a scary story. Be sure to specify that the story should be no longer than 10 sentences

Story: As an AI language model, I am not capable of experiencing emotions such as fear. However, I can generate a scary story for you. Here it is:

The old abandoned house at the end of the street had always been a source of curiosity for the neighborhood kids. One day, a group of them decided to explore it. As they entered, they heard strange noises coming from upstairs. They cautiously climbed the stairs, only to find a room with a single chair facing a wall. Suddenly, the door slammed shut behind them, and they were trapped. The chair slowly turned around, revealing a ghostly figure with glowing eyes. The kids screamed and tried to escape, but the door wouldn't budge. The ghostly figure approached them, and they could feel its icy breath on their faces. Just as they thought it was all over, the figure disappeared, and the door opened. The kids ran out of the house, neve

# Les Mis

In [45]:
initial_input = 'A French man spends 10 years and prison. Upen getting released, he has an experience that makes him believe in redemption'
optimize_for = 'beauty, emotion, and literary quality'

df_les_mis = generate_stories(
    initial_input=initial_input,
    optimize_for=optimize_for,
    llm=llm_language,
    n_iterations=10
)

print_first_and_best(df_les_mis)

  0%|          | 0/10 [00:00<?, ?it/s]

First Story

Prompt: A French man spends 10 years and prison. Upen getting released, he has an experience that makes him believe in redemption

Story: .

After spending a decade in prison, the French man was finally released. He had lost all hope of ever being able to live a normal life again. He had committed a crime in his youth and had paid the price for it. But now, he was a changed man. He had spent years reflecting on his actions and had come to the realization that he needed to make amends for what he had done.

As he walked out of the prison gates, he felt a sense of freedom that he had never experienced before. He knew that he had a long road ahead of him, but he was determined to make the most of his second chance.

The first few weeks were difficult. He struggled to find a job and to adjust to life outside of prison. But then, something amazing happened. He met a group of people who were dedicated to helping ex-convicts reintegrate into society. They offered him a job and a 

# Beautiful Entrance

In [50]:
initial_input = 'A beautiful woman stumbles into her physics lecture late on a rainy day. A young man notices her and has an idea.'
optimize_for = 'subtle beauty, emotion, story arc'

df_beautiful_entrance = generate_stories(
    initial_input=initial_input,
    optimize_for=optimize_for,
    llm=llm_language,
    n_iterations=10
)

print_first_and_best(df_beautiful_entrance)

  0%|          | 0/10 [00:00<?, ?it/s]

First Story

Prompt: A beautiful woman stumbles into her physics lecture late on a rainy day. A young man notices her and has an idea.

Story: As the beautiful woman stumbled into the physics lecture, the young man couldn't help but notice her. Her hair was drenched from the rain, and her clothes clung to her curves in all the right places. He couldn't take his eyes off her.

Suddenly, an idea struck him. He quickly scribbled a note on a piece of paper and passed it to the woman as she sat down.

The woman looked at the note and smiled. It read, "I couldn't help but notice how beautiful you are. Would you like to grab a coffee with me after class?"

The woman nodded and passed the note back to the young man. He grinned from ear to ear, feeling a rush of excitement.

As the lecture went on, the young man couldn't concentrate on anything the professor was saying. He was too busy thinking about the beautiful woman sitting next to him and the coffee date they had planned.

Finally, the lec